<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/Cross-Domain/CycleGAN_SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CycleGAN MNIST - SVHN 교차 도메인


이전 Coloration Cycle GAN 에서 Data Set 의 종류만 바꾼 코드이다. Instance Normalization 을 사용하기 위해서는 tensorflow-addons 을 설치해야 한다.

Data 처리 부분과 CycleGAN 의 모델 설정 부분 만 변환한 케이스이다. 

In [ ]:
import numpy as np
import os
import sys
from tensorflow.keras.layers import Input, Dropout, concatenate, add
from tensorflow.keras.layers import Conv2DTranspose, Conv2D 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LeakyReLU, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy import io

import cv2
import math
import datetime


#GPU 할당

In [ ]:
import tensorflow as tf 
physical_devices =tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

#유틸 함수 정의

In [ ]:
def display_images(imgs, filename, title='', imgs_dir=None, show=False):
  
    #이미지를 nxn 으로 나타냄
    rows = imgs.shape[1]
    cols = imgs.shape[2]
    channels = imgs.shape[3]
    side = int(math.sqrt(imgs.shape[0]))
    assert int(side * side) == imgs.shape[0]

    # 이미지 저장을 위한 폴더를 만듦
    if imgs_dir is None:
        imgs_dir = 'saved_images'
    save_dir = os.path.join(os.getcwd(), imgs_dir)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filename = os.path.join(imgs_dir, filename)
    # 이미지의 shape 을 지정
    if channels==1:
        imgs = imgs.reshape((side, side, rows, cols))
    else:
        imgs = imgs.reshape((side, side, rows, cols, channels))
    imgs = np.vstack([np.hstack(i) for i in imgs])
    plt.figure()
    plt.axis('off')
    plt.title(title)
    if channels==1:
        plt.imshow(imgs, interpolation='none', cmap='gray')
    else:
        plt.imshow(imgs, interpolation='none')
    plt.savefig(filename)
    if show:
        plt.show()
    
    plt.close('all')


def test_generator(generators, test_data, step, titles, dirs, todisplay=100, show=False):
    # generator 모델을 테스트함
    # 입력 인수
    """
    generators (tuple): 소스와 타겟 생성기
    test_data (tuple): 소스와 타겟 데이터
    step (int): 진행 단계
    titles (tuple): 표시 이미지의 타이틀
    dirs (tuple): 이미지 저장 폴더
    todisplay (int): 저장이미지의 수 (정사각형 형태로 생성되어야 한다.)
    show (bool): 이미지 표시 여부
    """

    # test data 로 부터 output 예측
    g_source, g_target = generators
    test_source_data, test_target_data = test_data
    t1, t2, t3, t4 = titles
    title_pred_source = t1
    title_pred_target = t2
    title_reco_source = t3
    title_reco_target = t4
    dir_pred_source, dir_pred_target = dirs

    pred_target_data = g_target.predict(test_source_data)
    pred_source_data = g_source.predict(test_target_data)
    reco_source_data = g_source.predict(pred_target_data)
    reco_target_data = g_target.predict(pred_source_data)

    # 정사각형 형태의 하나의 이미지로 나타냄
    imgs = pred_target_data[:todisplay]
    filename = '%06d.png' % step
    step = " Step: {:,}".format(step)
    title = title_pred_target + step
    display_images(imgs,
                   filename=filename,
                   imgs_dir=dir_pred_target,
                   title=title,
                   show=show)

    imgs = pred_source_data[:todisplay]
    title = title_pred_source
    display_images(imgs,
                   filename=filename,
                   imgs_dir=dir_pred_source,
                   title=title,
                   show=show)

    imgs = reco_source_data[:todisplay]
    title = title_reco_source
    filename = "reconstructed_source.png"
    display_images(imgs,
                   filename=filename,
                   imgs_dir=dir_pred_source,
                   title=title,
                   show=show)

    imgs = reco_target_data[:todisplay]
    title = title_reco_target
    filename = "reconstructed_target.png"
    display_images(imgs,
                   filename=filename,
                   imgs_dir=dir_pred_target,
                   title=title,
                   show=show)


def loaded_data(data, titles, filenames, todisplay=100):
    source_data, target_data, test_source_data, test_target_data = data
    test_source_filename, test_target_filename = filenames
    test_source_title, test_target_title = titles

    # 테스트 타겟 이미지 표시
    imgs = test_target_data[:todisplay]
    display_images(imgs,
                   filename=test_target_filename,
                   title=test_target_title)

    # 테스트 소스이미지 표시
    imgs = test_source_data[:todisplay]
    display_images(imgs,
                   filename=test_source_filename,
                   title=test_source_title)

    # 이미지 표시 정리
    target_data = target_data.astype('float32')  / 255
    test_target_data = test_target_data.astype('float32') / 255

    source_data = source_data.astype('float32')  / 255
    test_source_data = test_source_data.astype('float32') / 255

    # 소스, 타겟, 테스트 데이터
    data = (source_data, target_data, test_source_data, test_target_data)

    rows = source_data.shape[1]
    cols = source_data.shape[2]
    channels = source_data.shape[3]
    source_shape = (rows, cols, channels)

    rows = target_data.shape[1]
    cols = target_data.shape[2]
    channels = target_data.shape[3]
    target_shape = (rows, cols, channels)

    shapes = (source_shape, target_shape)
    
    return data, shapes

# 모델 함수 정의


In [ ]:
#모델 구성 단위 블럭 정의
def encoder_layer(inputs, filters=16, kernel_size=3, strides=2, activation='relu', instance_norm=True):
  # Conv2D - IN - LeakyReLU 로 구성된 일반 인코더 계층을 구성한다.
  conv = Conv2D(filters=filters,kernel_size=kernel_size, strides=strides, padding='same')
  x = inputs
  if instance_norm:
    x = InstanceNormalization()(x)
  if activation == 'relu':
    x = Activation('relu')(x)
  else:
    x = LeakyReLU(alpha=0.2)(x)
  
  x = conv(x)
  return x

def decoder_layer(inputs, paired_inputs, filters=16, kernel_size=3, strides=2,activation='relu', instance_norm=True):
  #Conv2DTranspose-IN-LeakyReLU로 구성된 디코더 계층구성, 활성화 함수는 ReLU 로 교체될 수 있음
  #paired_inputs 는 U-net 의 skip connection 을 의미하며 입력에 연결된다.

  conv=Conv2DTranspose(filters=filters,kernel_size=kernel_size, strides=strides, padding='same')
  
  x = inputs
  if instance_norm:
    x = InstanceNormalization()(x)
  if activation=='relu':
    x = Activation('relu')(x)
  else:
    x = LeakyReLU(alpha=0.2)(x)
  
  x = conv(x)
  x = concatenate([x, paired_inputs])
  return x

In [ ]:
#생성기 U-Net Build
def build_generator(input_shape, output_shape=None, kernel_size=3, name=None):
  # 4계층 인코더와 4계층 디코더로 구성된 U-Net 을 구성한다.

  inputs = Input(shape=input_shape)
  channels = int(output_shape[-1])

  e1 = encoder_layer(inputs, filters=32, kernel_size=kernel_size, activation='leaky_relu', strides=1)
  e2 = encoder_layer(e1, filters=64, kernel_size= kernel_size, activation='leaky_relu')
  e3 = encoder_layer(e2, filters=128, kernel_size= kernel_size, activation='leaky_relu')
  e4 = encoder_layer(e3, filters=128, kernel_size= kernel_size, activation='leaky_relu')

  d1 = decoder_layer(e4,e3,filters=128,kernel_size=kernel_size)
  d2 = decoder_layer(d1,e2,filters=64,kernel_size=kernel_size)
  d3 = decoder_layer(d2,e1,filters=64,kernel_size=kernel_size)

  outputs = Conv2DTranspose(channels, kernel_size=kernel_size, strides=1, activation='sigmoid',padding='same')(d3)

  generator = Model(inputs, outputs, name=name)
  
  return generator

#PatchGAN Discriminator Build
def build_discriminator(input_shape,kernel_size=3, patchgan=True,name=None):
  inputs = Input(shape=input_shape)
  x = encoder_layer(inputs,32,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 64,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 128,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)
  x = encoder_layer(x, 256,kernel_size=kernel_size,activation='leaky_relu',instance_norm=False)

  #patchgan=True 이면 n x n 차원 확률 출력 사용
  if patchgan:
    x = LeakyReLU(alpha=0.2)(x)
    outputs = Conv2D(1, kernel_size=kernel_size, strides=1, padding='same')(x)
  else:
    x = Flatten()(x)
    x = Dense(1)(x)
    outputs - Activation('linear')(x)

  discriminator = Model(inputs, outputs, name=name)

  return discriminator

# CycleGAN Build 함수 지정

아래에서 동질성 Loss identify 는 색상을 재현하는데 있어, 발생한 문제에 대해 본래의 색상을 찾아가도록 훈련시키는 과정이다. 이를 위해 타겟데이터 y 가 들어왔을 때 본래 이미지로 재구성할 수 있는 능력을 훈련시키는 것이다.

In [ ]:
def build_cyclegan(shape, source_name='source', target_name='target', kernel_size=3, patchgan=False, identify=False):
  #Cycle GAN 의 구성
  """
  1) 타깃과 소스의 판별기
  2) 타깃과 소스의 생성기
  3) 적대적 네트워크 구성
  """

  #입력 인수
  """
  shape(tuple): 소스와 타깃 형상
  source_name (string): 판별기/생성기 모델 이름 뒤에 붙는 소스이름 문자열
  target_name (string): 판별기/생성기 모델 이름 뒤에 붙는 타깃이름 문자열
  target_size (int): 인코더/디코더 혹은 판별기/생성기 모델에 사용될 커널 크기
  patchgan(bool): 판별기에 patchgan 사용 여부
  identify(bool): 동질성 사용 여부
  """
  #출력 결과:
  #list : 2개의 생성기, 2개의 판별기, 1개의 적대적 모델 

  source_shape, target_shape = shapes
  lr = 2e-4
  decay = 6e-8
  gt_name = "gen_" + target_name
  gs_name = "gen_" + source_name
  dt_name = "dis_" + target_name
  ds_name = "dis_" + source_name

  #타깃과 소스 생성기 구성
  g_target = build_generator(source_shape, target_shape, kernel_size=kernel_size, name=gt_name)
  g_source = build_generator(target_shape, source_shape, kernel_size=kernel_size, name=gs_name)

  print('-----Target Generator-----')
  g_target.summary()
  print('-----Source Generator-----')
  g_source.summary()

  #타깃과 소스 판별기 구성
  d_target = build_discriminator(target_shape, patchgan=patchgan, kernel_size=kernel_size,name=dt_name)
  d_source = build_discriminator(source_shape, patchgan=patchgan, kernel_size=kernel_size,name=ds_name)

  print('-----Targent Discriminator-----')
  d_target.summary()
  print('-----Source Discriminator-----')
  d_source.summary()

  optimizer = RMSprop(lr=lr, decay=decay)
  d_target.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
  d_source.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

  #적대적 모델에서 판별기 가중치 고정
  d_target.trainable=False
  d_source.trainable=False

  #적대적 모델의 계산 그래프 구성

  #전방순환 네트워크와 타깃 판별기 
  source_input = Input(shape=source_shape)
  fake_target = g_target(source_input)
  preal_target = d_target(fake_target)
  reco_source = g_source(fake_target)

  #후방순환 네트워크와 타깃 판별기
  target_input = Input(shape=target_shape)
  fake_source = g_source(target_input)
  preal_source = d_source(fake_source)
  reco_target = g_target(fake_source)

  #동질성 손실 사용 시, 두개의 손실항과 출력을 추가한다.
  if identify:
    iden_source = g_source(source_input)
    iden_target = g_target(target_input)
    loss = ['mse', 'mse', 'mae', 'mae', 'mae', 'mae']
    loss_weight = [1.,1.,10.,10.,0.5,0.5]
    inputs = [source_input, target_input]
    outputs = [preal_source, preal_target, reco_source, reco_target, iden_source, iden_target]

  else:
    loss = ['mse', 'mse', 'mae', 'mae']
    loss_weights = [1., 1., 10., 10.]
    inputs = [source_input, target_input]
    outputs = [preal_source, preal_target, reco_source, reco_target]

  #적대적 모델 구성
  adv = Model(inputs, outputs, name='adversarial')
  optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
  adv.compile(loss=loss, loss_weights=loss_weights, optimizer=optimizer,metrics=['accuracy'])
  print('-----Adversarial Network-----')
  adv.summary()

  return g_source, g_target, d_source, d_target, adv

# 모델 훈련 함수 지정

In [ ]:
def train_cyclegan(models, data, params, test_params, test_generator):
  #Cycle GAN 훈련
  """
  1) 타깃 판별기 훈련
  2) 소스 판별기 훈련
  3) 적대적 네트워크의 전방/ 후방 순환 훈련
  """
  # 입력 인수:
  """
  models (list): 소스/타깃에 대한 판별기/생성기, 적대적 모델
  data (tuple): 소스와 타깃 훈련데이터
  params (tuple): 네트워크 매개변수
  test_params (tuple): 테스트 매개변수
  test_generator (function): 예측 타깃/소스 이미지생성에 사용됨.
  """
  #모델
  g_source, g_target, d_source, d_target, adv = models
  #네트워크 매개변수 지정
  batch_size, train_steps, patch, model_name = params
  #훈련 데이터 세트 
  source_data, target_data, test_source_data, test_target_data = data
  titles, dirs = test_params

  #생성기 이미지는 2000단계마다 저장됨
  save_interval = 2000
  target_size = target_data.shape[0]
  source_size = source_data.shape[0]

  #patchgan 사용 여부
  if patch > 1:
    d_patch = (patch, patch, 1)
    valid = np.ones((batch_size,) + d_patch)
    fake = np.zeros((batch_size,) + d_patch)

  else:
    valid = np.ones([batch_size, 1])
    fake = np.zeros([batch_size, 1])

  valid_fake = np.concatenate((valid,fake))
  start_time = datetime.datetime.now()

  for step in range(train_steps):
    #실제 타깃 데이터 배치 샘플링
    rand_indices = np.random.randint(0, target_size, size=batch_size)
    real_target = target_data[rand_indices]

    #실제 소스 데이터 배치 샘플링
    rand_indices = np.random.randint(0, source_size, size=batch_size)
    real_source = source_data[rand_indices]

    #실제 소스 데이터에서 가짜 타깃 데이터 배치를 생성
    fake_target = g_target.predict(real_source)
    
    #실제 타겟 데이터와 가짜 타겟데이터를 하나의 배치로 결합
    x = np.concatenate((real_target, fake_target))
    #타겟 판별자를 훈련시킴
    metrics = d_target.train_on_batch(x, valid_fake)
    log = "%d: [d_target loss: %f]" %(step, metrics[0])

    #실제 타깃 데이터에서 가짜 소스 데이터 배치 생성
    fake_source = g_source.predict(real_target)
    x = np.concatenate((real_source, fake_source))
    #가짜/실제 데이터를 사용해 소스 판별기 훈련
    metrics = d_source.train_on_batch(x, valid_fake)
    log = "%s [d_source loss: %f]" %(log, metrics[0])

    #전방/후방 순환을 사용해 적대적 네트워크 훈련
    #생성된 가짜 소스/타깃 데이터는 판별기를 속이려고 시도함
    x = [real_source, real_target]
    y = [valid, valid, real_source, real_target]

    metrics = adv.train_on_batch(x, y)

    elapsed_time = datetime.datetime.now()-start_time
    fmt = "%s [adv loss: %f] [time: %s]"
    log = fmt %(log, metrics[0], elapsed_time)
    print(log)

    if (step+1) % save_interval == 0:
      if (step+1) == train_steps:
        show = True
      else:
        show = False

      test_generator((g_source, g_target), (test_source_data, test_target_data), step = step+1, titles=titles, dirs=dirs, show=show)

  g_source.save(model_name + "-g_source.h5")
  g_target.save(model_name + "-g_target.h5")

#SVHN 을 위한 데이터 로딩 함수지정

In [ ]:
def get_datadir():
    cache_dir = os.path.join(os.path.expanduser('~'), '.keras')
    cache_subdir = 'datasets'
    datadir_base = os.path.expanduser(cache_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.path.join('/tmp', '.keras')

    datadir = os.path.join(datadir_base, cache_subdir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    return datadir

def loadmat(filename):
    # SVHN 데이터셋을 로딩한다.
    mat = io.loadmat(filename)
    # 이미지 데이터의 키는 'X' 이고 ,이미지 라벨의 키는 'y' 로 구성되어 있다. 
    # 본 예제에세는 X 만을 사용한다.
    data = mat['X']
    rows =data.shape[0]
    cols = data.shape[1]
    channels = data.shape[2]
    # 매트랩 데이터에서, 이미지의 인덱스는 마지막에 지정된다.
    # 케라스에서, 이미지의 인덱스는 첫번째로 지정된다.
    # 케라스의 방식으로 매트랩 데이터를 바꾸어준다.
    data = np.transpose(data, (3, 0, 1, 2))
    return data


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import get_file

In [ ]:
#MNIST 데이터 로딩 및 전처리
(source_data, _), (test_source_data, _) = mnist.load_data()

#28x28 사이즈의 MNIST 데이터를 SVHN 의 32x32로 변환하기위한 패드를 지정한다.
source_data = np.pad(source_data, ((0,0), (2,2), (2,2)),'constant', constant_values=0)
test_source_data = np.pad(test_source_data, ((0,0), (2,2), (2,2)), 'constant', constant_values=0)

#Input 이미지의 차원을 data format을 채널값이 마지막 값인것으로 유추하여 지정한다.
rows = source_data.shape[1]
cols = source_data.shape[2]
channels = 1

# 이미지의 크기를 rowxcolxchannels 의 CNN 이미지를 위해 재정의 한다.
size = source_data.shape[0]
source_data = source_data.reshape(size, rows, cols, channels)
size = test_source_data.shape[0]
test_source_data = test_source_data.reshape(size, rows, cols, channels)

In [ ]:
#SVHN 데이터 로딩 및 전처리
datadir = get_datadir()
get_file('train_32x32.mat', origin='http://ufldl.stanford.edu/housenumbers/train_32x32.mat')
get_file('test_32x32.mat', 'http://ufldl.stanford.edu/housenumbers/test_32x32.mat')
path = os.path.join(datadir, 'train_32x32.mat')
target_data = loadmat(path)
path = os.path.join(datadir, 'test_32x32.mat')
test_target_data = loadmat(path)

# 소스 데이터, 타겟데이터, 테스트 데이터 지정
data = (source_data, target_data, test_source_data, test_target_data)
filenames = ('mnist_test_source.png', 'svhn_test_target.png')
titles = ('MNIST test source images', 'SVHN test target images')

In [ ]:
# 이미지 저장 및 shape 재지정
data, shapes = loaded_data(data, titles, filenames)

# 모델 Parameter 설정 및 빌드

In [ ]:
model_name = 'cyclegan_svhn'
batch_size = 32
train_steps = 100000
patchgan = True
kernel_size = 5
postfix = ('%dp' % kernel_size) if patchgan else ('%d' % kernel_size)

In [ ]:
titles = ('MNIST predicted source images.', 'SVHN predicted target images.', 'MNIST reconstructed source images.', 'SVHN reconstructed target images.')
dirs = ('mnist_source-%s' % postfix, 'svhn_target-%s' % postfix)

In [ ]:
models = build_cyclegan(shapes, "gray-%s" % postfix, "color-%s" % postfix, kernel_size=kernel_size, patchgan=patchgan)

-----Target Generator-----
Model: "gen_color-5p"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
instance_normalization (Instanc (None, 32, 32, 1)    2           input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 32, 32, 1)    0           instance_normalization[0][0]     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   832         leaky_re_lu[0][0]                
____________________________________________________________

In [ ]:
#판별기의 입력을 2^n 만큼 척도를 줄임 -> patch 크기를 2^n 으로 나눔(즉 strides=2를 n회 사용함)
patch = int(source_data.shape[1] / 2**4) if patchgan else 1
params = (batch_size, train_steps, patch, model_name)
test_params = (titles, dirs)

#cyclegan 훈련
train_cyclegan(models, data, params, test_params, test_generator)